The UEFA Champions League, often called the Champions League, is a preeminent annual soccer competition that captivates fans worldwide. Established in 1955 as the European Champion Clubs' Cup, it evolved into the UEFA Champions League in 1992, broadening its appeal. The modern format features 32 top-tier club teams selected based on their domestic league performance, adding to the intrigue.


![Stadium.jpg](Stadium.jpg)


This electrifying event transcends sports, becoming a celebration of unity, culture, and national pride. Fans, draped in their countries' colors, create an electric atmosphere, making the tournament as much about the spectators as the players.Financially, the Champions League is a lifeline for clubs, boosting revenues and offering transformative opportunities. Nevertheless, it sparks debates about wealth disparities in European soccer.



![Duel_football](Duel_football.jpg)

The Champions League is synonymous with historic rivalries, underdog triumphs, and individual brilliance. For players, it represents a career pinnacle, while for fans, it's a cultural phenomenon. The iconic anthem and rituals enrich the soccer experience. In 200 words, the UEFA Champions League is the epitome of European soccer excellence, offering unforgettable moments, financial rewards, and a unique cultural impact, with 32 top clubs adding to its allure.






## Schema name: `SOCCER`
## Table Name(s): `TBL_UEFA_2020` | `TBL_UEFA_2021` | `TBL_UEFA_2022`
### Note : All three tables have same column names and data types

| Column | Definition | Data type |
|--------|------------|-----------|
| `STAGE`| Stage of the March | `VARCHAR(50)` |
| `DATE` | When the match occurred. | `DATE` |
| `PENS` | Did the match end with penalty | `VARCHAR(50)` |
| `PENS_HOME_SCORE` | In case of penalty, score by home team | `VARCHAR(50)` |
| `PENS_AWAY_SCORE` | In case of penalty, score by away team | `VARCHAR(50)` |
| `TEAM_NAME_HOME` | Team home name | `VARCHAR(50)` |
| `TEAM_NAME_AWAY`| Team away  name | `VARCHAR(50)` |
| `TEAM_HOME_SCORE` | Team home score | `NUMBER` |
| `TEAM_AWAY_SCORE` | Team away score | `NUMBER` |
| `POSSESSION_HOME` | Ball possession for the home team | `FLOAT` |
| `POSSESSION_AWAY` | Ball possession for the away team | `FLOAT` |
| `TOTAL_SHOTS_HOME` | Number of shots by the home team | `NUMBER` |
| `TOTAL_SHOTS_AWAY` | Number of shots by the away team | `NUMBER`
| `SHOTS_ON_TARGET_HOME` | Total shot for home team | `FLOAT` |
| `SHOTS_ON_TARGET_AWAY` | Total shot for away team | `FLOAT` |
| `DUELS_WON_HOME` | duel win possession of ball - for home team | `NUMBER` |
| `DUELS_WON_AWAY` | duel win possession of ball - for away team | `NUMBER` 
| `PREDICTION_TEAM_HOME_WIN` | Probability of home team to win | `FLOAT` |
| `PREDICTION_DRAW` | Probability of draw | `FLOAT` |
| `PREDICTION_TEAM_AWAY_WIN` | Probability of away team to win | `FLOAT` |
| `LOCATION` | Stadium where the match was held | `VARCHAR(50)` | 
Note that *in Snowflake all databases, tables, and columns are **upper case*** by default.

You will execute SQL queries to answer three questions, as listed in the instructions.

In [1]:
-- TEAM_HOME_WITH_MOST_GOALS
SELECT 
    TEAM_NAME_HOME AS HOME_TEAM,
    SUM(TEAM_HOME_SCORE) AS TOTAL_HOME_GOALS
FROM (
    SELECT TEAM_NAME_HOME, TEAM_HOME_SCORE FROM SOCCER.TBL_UEFA_2020
    UNION ALL
    SELECT TEAM_NAME_HOME, TEAM_HOME_SCORE FROM SOCCER.TBL_UEFA_2021
    UNION ALL
    SELECT TEAM_NAME_HOME, TEAM_HOME_SCORE FROM SOCCER.TBL_UEFA_2022
)
GROUP BY TEAM_NAME_HOME
ORDER BY TOTAL_HOME_GOALS DESC
LIMIT 1;

,HOME_TEAM,TOTAL_HOME_GOALS
0,Real Madrid,26


In [2]:
-- TEAM_WITH_MAJORITY_POSSESSION
WITH TeamPossession AS (
    -- Home teams with majority possession
    SELECT 
        TEAM_NAME_HOME AS TEAM_NAME,
        COUNT(*) AS MAJORITY_POSSESSION_MATCHES
    FROM SOCCER.TBL_UEFA_2020
    WHERE POSSESSION_HOME > 50
    GROUP BY TEAM_NAME_HOME
    
    UNION ALL
    
    SELECT 
        TEAM_NAME_HOME AS TEAM_NAME,
        COUNT(*) AS MAJORITY_POSSESSION_MATCHES
    FROM SOCCER.TBL_UEFA_2021
    WHERE POSSESSION_HOME > 50
    GROUP BY TEAM_NAME_HOME
    
    UNION ALL
    
    SELECT 
        TEAM_NAME_HOME AS TEAM_NAME,
        COUNT(*) AS MAJORITY_POSSESSION_MATCHES
    FROM SOCCER.TBL_UEFA_2022
    WHERE POSSESSION_HOME > 50
    GROUP BY TEAM_NAME_HOME
    
    UNION ALL
    
    -- Away teams with majority possession
    SELECT 
        TEAM_NAME_AWAY AS TEAM_NAME,
        COUNT(*) AS MAJORITY_POSSESSION_MATCHES
    FROM SOCCER.TBL_UEFA_2020
    WHERE POSSESSION_AWAY > 50
    GROUP BY TEAM_NAME_AWAY
    
    UNION ALL
    
    SELECT 
        TEAM_NAME_AWAY AS TEAM_NAME,
        COUNT(*) AS MAJORITY_POSSESSION_MATCHES
    FROM SOCCER.TBL_UEFA_2021
    WHERE POSSESSION_AWAY > 50
    GROUP BY TEAM_NAME_AWAY
    
    UNION ALL
    
    SELECT 
        TEAM_NAME_AWAY AS TEAM_NAME,
        COUNT(*) AS MAJORITY_POSSESSION_MATCHES
    FROM SOCCER.TBL_UEFA_2022
    WHERE POSSESSION_AWAY > 50
    GROUP BY TEAM_NAME_AWAY
)

SELECT 
    TEAM_NAME,
    SUM(MAJORITY_POSSESSION_MATCHES) AS TOTAL_MAJORITY_POSSESSION_MATCHES
FROM TeamPossession
GROUP BY TEAM_NAME
ORDER BY TOTAL_MAJORITY_POSSESSION_MATCHES DESC
LIMIT 1;

,TEAM_NAME,TOTAL_MAJORITY_POSSESSION_MATCHES


In [3]:
-- TEAM_WON_DUEL_LOST_GAME_STAGE_WISE
WITH DuelWinnersMatchLosers AS (
    -- 2020 season
    SELECT 
        STAGE,
        TEAM_NAME_HOME AS TEAM_NAME,
        DUELS_WON_HOME AS DUELS_WON,
        TEAM_HOME_SCORE AS TEAM_SCORE,
        TEAM_AWAY_SCORE AS OPPONENT_SCORE,
        'HOME' AS TEAM_TYPE
    FROM SOCCER.TBL_UEFA_2020
    WHERE DUELS_WON_HOME > DUELS_WON_AWAY AND TEAM_HOME_SCORE < TEAM_AWAY_SCORE
    
    UNION ALL
    
    SELECT 
        STAGE,
        TEAM_NAME_AWAY AS TEAM_NAME,
        DUELS_WON_AWAY AS DUELS_WON,
        TEAM_AWAY_SCORE AS TEAM_SCORE,
        TEAM_HOME_SCORE AS OPPONENT_SCORE,
        'AWAY' AS TEAM_TYPE
    FROM SOCCER.TBL_UEFA_2020
    WHERE DUELS_WON_AWAY > DUELS_WON_HOME AND TEAM_AWAY_SCORE < TEAM_HOME_SCORE
    
    UNION ALL
    
    -- 2021 season
    SELECT 
        STAGE,
        TEAM_NAME_HOME AS TEAM_NAME,
        DUELS_WON_HOME AS DUELS_WON,
        TEAM_HOME_SCORE AS TEAM_SCORE,
        TEAM_AWAY_SCORE AS OPPONENT_SCORE,
        'HOME' AS TEAM_TYPE
    FROM SOCCER.TBL_UEFA_2021
    WHERE DUELS_WON_HOME > DUELS_WON_AWAY AND TEAM_HOME_SCORE < TEAM_AWAY_SCORE
    
    UNION ALL
    
    SELECT 
        STAGE,
        TEAM_NAME_AWAY AS TEAM_NAME,
        DUELS_WON_AWAY AS DUELS_WON,
        TEAM_AWAY_SCORE AS TEAM_SCORE,
        TEAM_HOME_SCORE AS OPPONENT_SCORE,
        'AWAY' AS TEAM_TYPE
    FROM SOCCER.TBL_UEFA_2021
    WHERE DUELS_WON_AWAY > DUELS_WON_HOME AND TEAM_AWAY_SCORE < TEAM_HOME_SCORE
    
    UNION ALL
    
    -- 2022 season
    SELECT 
        STAGE,
        TEAM_NAME_HOME AS TEAM_NAME,
        DUELS_WON_HOME AS DUELS_WON,
        TEAM_HOME_SCORE AS TEAM_SCORE,
        TEAM_AWAY_SCORE AS OPPONENT_SCORE,
        'HOME' AS TEAM_TYPE
    FROM SOCCER.TBL_UEFA_2022
    WHERE DUELS_WON_HOME > DUELS_WON_AWAY AND TEAM_HOME_SCORE < TEAM_AWAY_SCORE
    
    UNION ALL
    
    SELECT 
        STAGE,
        TEAM_NAME_AWAY AS TEAM_NAME,
        DUELS_WON_AWAY AS DUELS_WON,
        TEAM_AWAY_SCORE AS TEAM_SCORE,
        TEAM_HOME_SCORE AS OPPONENT_SCORE,
        'AWAY' AS TEAM_TYPE
    FROM SOCCER.TBL_UEFA_2022
    WHERE DUELS_WON_AWAY > DUELS_WON_HOME AND TEAM_AWAY_SCORE < TEAM_HOME_SCORE
)

SELECT 
    STAGE,
    TEAM_NAME,
    COUNT(*) AS OCCURRENCES,
    SUM(DUELS_WON) AS TOTAL_DUELS_WON,
    AVG(DUELS_WON) AS AVG_DUELS_PER_MATCH,
    SUM(TEAM_SCORE) AS TOTAL_GOALS_SCORED,
    SUM(OPPONENT_SCORE) AS TOTAL_GOALS_CONCEDED
FROM DuelWinnersMatchLosers
GROUP BY STAGE, TEAM_NAME
ORDER BY STAGE, OCCURRENCES DESC;

,STAGE,TEAM_NAME,OCCURRENCES,TOTAL_DUELS_WON,AVG_DUELS_PER_MATCH,TOTAL_GOALS_SCORED,TOTAL_GOALS_CONCEDED
0,Final,Inter,1,0.64,0.64,0,1
1,Group stage: Matchday 1,Marseille,1,0.55,0.55,0,2
2,Group stage: Matchday 1,København,1,0.51,0.51,0,3
3,Group stage: Matchday 1,Milan,1,0.62,0.62,2,3
4,Group stage: Matchday 1,Leipzig,1,0.58,0.58,3,6
...,...,...,...,...,...,...,...
138,Semi-finals second leg,Real Madrid,1,0.59,0.59,0,4
139,Semi-finals second leg,Milan,1,0.55,0.55,0,1
140,Semi-finals second leg,Man City,1,0.53,0.53,1,3
141,Group stage: Matchday 6,Sevilla,1,0.51,0.51,1,3


In [1]:
-- TEAMS_WITH_HIGHEST_WIN_RATES
WITH MatchResults AS (
    SELECT 
        TEAM_NAME_HOME AS TEAM_NAME,
        CASE WHEN TEAM_HOME_SCORE > TEAM_AWAY_SCORE THEN 1 ELSE 0 END AS WINS,
        1 AS MATCHES
    FROM SOCCER.TBL_UEFA_2020
    UNION ALL
    SELECT 
        TEAM_NAME_AWAY AS TEAM_NAME,
        CASE WHEN TEAM_AWAY_SCORE > TEAM_HOME_SCORE THEN 1 ELSE 0 END AS WINS,
        1 AS MATCHES
    FROM SOCCER.TBL_UEFA_2020
    UNION ALL
    SELECT 
        TEAM_NAME_HOME AS TEAM_NAME,
        CASE WHEN TEAM_HOME_SCORE > TEAM_AWAY_SCORE THEN 1 ELSE 0 END AS WINS,
        1 AS MATCHES
    FROM SOCCER.TBL_UEFA_2021
    UNION ALL
    SELECT 
        TEAM_NAME_AWAY AS TEAM_NAME,
        CASE WHEN TEAM_AWAY_SCORE > TEAM_HOME_SCORE THEN 1 ELSE 0 END AS WINS,
        1 AS MATCHES
    FROM SOCCER.TBL_UEFA_2021
    UNION ALL
    SELECT 
        TEAM_NAME_HOME AS TEAM_NAME,
        CASE WHEN TEAM_HOME_SCORE > TEAM_AWAY_SCORE THEN 1 ELSE 0 END AS WINS,
        1 AS MATCHES
    FROM SOCCER.TBL_UEFA_2022
    UNION ALL
    SELECT 
        TEAM_NAME_AWAY AS TEAM_NAME,
        CASE WHEN TEAM_AWAY_SCORE > TEAM_HOME_SCORE THEN 1 ELSE 0 END AS WINS,
        1 AS MATCHES
    FROM SOCCER.TBL_UEFA_2022
)

SELECT 
    TEAM_NAME,
    SUM(WINS) AS TOTAL_WINS,
    SUM(MATCHES) AS TOTAL_MATCHES,
    ROUND(SUM(WINS) * 100.0 / SUM(MATCHES), 2) AS WIN_PERCENTAGE
FROM MatchResults
GROUP BY TEAM_NAME
HAVING SUM(MATCHES) >= 5  -- Only include teams with at least 5 matches
ORDER BY WIN_PERCENTAGE DESC
LIMIT 10;

,TEAM_NAME,TOTAL_WINS,TOTAL_MATCHES,WIN_PERCENTAGE
0,Bayern Munich,8,10,80.00
1,Bayern Munich,8,10,80.00
2,Manchester City,11,14,78.57
3,Juventus,6,8,75.00
4,Chelsea,10,14,71.43
5,Napoli,7,10,70.00
6,Liverpool,14,20,70.00
7,Real Madrid,16,24,66.67
8,Porto,4,6,66.67
9,Bayern,6,9,66.67


In [2]:
-- STADIUMS_WITH_HIGHEST_SCORING_MATCHES
SELECT 
    LOCATION,
    COUNT(*) AS TOTAL_MATCHES,
    AVG(TEAM_HOME_SCORE + TEAM_AWAY_SCORE) AS AVG_GOALS_PER_MATCH
FROM (
    SELECT LOCATION, TEAM_HOME_SCORE, TEAM_AWAY_SCORE FROM SOCCER.TBL_UEFA_2020
    UNION ALL
    SELECT LOCATION, TEAM_HOME_SCORE, TEAM_AWAY_SCORE FROM SOCCER.TBL_UEFA_2021
    UNION ALL
    SELECT LOCATION, TEAM_HOME_SCORE, TEAM_AWAY_SCORE FROM SOCCER.TBL_UEFA_2022
)
GROUP BY LOCATION
HAVING COUNT(*) >= 3  -- Only include stadiums with at least 3 matches
ORDER BY AVG_GOALS_PER_MATCH DESC
LIMIT 10;

,LOCATION,TOTAL_MATCHES,AVG_GOALS_PER_MATCH
0,Nou Camp,3,5.000000
1,Ibrox Stadium,3,5.000000
2,Amsterdam Arena,3,4.666667
3,San Paolo,3,4.666667
4,Old Trafford,3,4.666667
5,Doosan Arena,3,4.666667
6,Red Bull Arena Salzburg,3,4.666667
7,Sammy Ofer,3,4.333333
8,BORUSSIA-PARK,3,4.333333
9,Olimpico,4,4.250000


In [3]:
-- TEAMS_WITH_BEST_CONVERSION_RATES
WITH TeamStats AS (
    -- Home team stats
    SELECT 
        TEAM_NAME_HOME AS TEAM_NAME,
        TEAM_HOME_SCORE AS GOALS,
        TOTAL_SHOTS_HOME AS SHOTS,
        SHOTS_ON_TARGET_HOME AS SHOTS_ON_TARGET
    FROM SOCCER.TBL_UEFA_2020
    UNION ALL
    SELECT 
        TEAM_NAME_HOME AS TEAM_NAME,
        TEAM_HOME_SCORE AS GOALS,
        TOTAL_SHOTS_HOME AS SHOTS,
        SHOTS_ON_TARGET_HOME AS SHOTS_ON_TARGET
    FROM SOCCER.TBL_UEFA_2021
    UNION ALL
    SELECT 
        TEAM_NAME_HOME AS TEAM_NAME,
        TEAM_HOME_SCORE AS GOALS,
        TOTAL_SHOTS_HOME AS SHOTS,
        SHOTS_ON_TARGET_HOME AS SHOTS_ON_TARGET
    FROM SOCCER.TBL_UEFA_2022
    UNION ALL
    -- Away team stats
    SELECT 
        TEAM_NAME_AWAY AS TEAM_NAME,
        TEAM_AWAY_SCORE AS GOALS,
        TOTAL_SHOTS_AWAY AS SHOTS,
        SHOTS_ON_TARGET_AWAY AS SHOTS_ON_TARGET
    FROM SOCCER.TBL_UEFA_2020
    UNION ALL
    SELECT 
        TEAM_NAME_AWAY AS TEAM_NAME,
        TEAM_AWAY_SCORE AS GOALS,
        TOTAL_SHOTS_AWAY AS SHOTS,
        SHOTS_ON_TARGET_AWAY AS SHOTS_ON_TARGET
    FROM SOCCER.TBL_UEFA_2021
    UNION ALL
    SELECT 
        TEAM_NAME_AWAY AS TEAM_NAME,
        TEAM_AWAY_SCORE AS GOALS,
        TOTAL_SHOTS_AWAY AS SHOTS,
        SHOTS_ON_TARGET_AWAY AS SHOTS_ON_TARGET
    FROM SOCCER.TBL_UEFA_2022
)

SELECT 
    TEAM_NAME,
    SUM(GOALS) AS TOTAL_GOALS,
    SUM(SHOTS) AS TOTAL_SHOTS,
    SUM(SHOTS_ON_TARGET) AS TOTAL_SHOTS_ON_TARGET,
    ROUND(SUM(GOALS) * 100.0 / NULLIF(SUM(SHOTS), 0), 2) AS SHOT_CONVERSION_RATE,
    ROUND(SUM(GOALS) * 100.0 / NULLIF(SUM(SHOTS_ON_TARGET), 0), 2) AS SHOT_ON_TARGET_CONVERSION_RATE
FROM TeamStats
GROUP BY TEAM_NAME
HAVING SUM(SHOTS) >= 50  -- Only include teams with at least 50 total shots
ORDER BY SHOT_CONVERSION_RATE DESC
LIMIT 10;

,TEAM_NAME,TOTAL_GOALS,TOTAL_SHOTS,TOTAL_SHOTS_ON_TARGET,SHOT_CONVERSION_RATE,SHOT_ON_TARGET_CONVERSION_RATE
0,B. Mâ€˜Gladbach,16,75,33,21.33,48.48
1,FC Porto,15,83,39,18.07,38.46
2,Bayern,26,148,54,17.57,48.15
3,Manchester United,15,86,36,17.44,41.67
4,RB Leipzig,13,77,29,16.88,44.83
5,Benfica,36,214,80,16.82,45.00
6,Bayern Munich,27,165,62,16.36,43.55
7,Manchester City,32,198,89,16.16,35.96
8,Napoli,26,162,68,16.05,38.24
9,Paris Saint-Germain,16,100,44,16.00,36.36


In [4]:
-- MATCH_STAGE_ANALYSIS
SELECT 
    STAGE,
    COUNT(*) AS TOTAL_MATCHES,
    AVG(TEAM_HOME_SCORE + TEAM_AWAY_SCORE) AS AVG_GOALS_PER_MATCH,
    AVG(POSSESSION_HOME) AS AVG_HOME_POSSESSION,
    AVG(POSSESSION_AWAY) AS AVG_AWAY_POSSESSION,
    SUM(CASE WHEN PENS IS NOT NULL THEN 1 ELSE 0 END) AS PENALTY_SHOOTOUTS,
    ROUND(SUM(CASE WHEN TEAM_HOME_SCORE > TEAM_AWAY_SCORE THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS HOME_WIN_PERCENTAGE,
    ROUND(SUM(CASE WHEN TEAM_HOME_SCORE < TEAM_AWAY_SCORE THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS AWAY_WIN_PERCENTAGE,
    ROUND(SUM(CASE WHEN TEAM_HOME_SCORE = TEAM_AWAY_SCORE THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS DRAW_PERCENTAGE
FROM (
    SELECT * FROM SOCCER.TBL_UEFA_2020
    UNION ALL
    SELECT * FROM SOCCER.TBL_UEFA_2021
    UNION ALL
    SELECT * FROM SOCCER.TBL_UEFA_2022
)
GROUP BY STAGE
ORDER BY 
    CASE 
        WHEN STAGE LIKE 'Group%' THEN 1
        WHEN STAGE LIKE 'Round of%' THEN 2
        WHEN STAGE LIKE 'Quarter%' THEN 3
        WHEN STAGE LIKE 'Semi%' THEN 4
        WHEN STAGE LIKE 'Final%' THEN 5
        ELSE 6
    END;

,STAGE,TOTAL_MATCHES,AVG_GOALS_PER_MATCH,AVG_HOME_POSSESSION,AVG_AWAY_POSSESSION,PENALTY_SHOOTOUTS,HOME_WIN_PERCENTAGE,AWAY_WIN_PERCENTAGE,DRAW_PERCENTAGE
0,Group stage: Matchday 6,6,4.333333,0.510000,0.490000,6,33.33,50.00,16.67
1,Final,4,1.000000,0.572500,0.427500,4,25.00,75.00,0.00
2,Semi-finals second leg,6,3.000000,0.445000,0.555000,6,83.33,16.67,0.00
3,Semi-finals first leg,6,3.000000,0.545000,0.455000,6,33.33,33.33,33.33
4,Quarter-finals second leg,12,2.500000,0.524167,0.475833,12,0.00,41.67,58.33
5,Quarter-finals first leg,12,2.666667,0.513333,0.486667,12,58.33,41.67,0.00
6,Group stage: Matchday 5,48,3.062500,0.480833,0.519167,48,41.67,35.42,22.92
7,Group stage: Matchday 4,48,3.125000,0.471875,0.526042,48,39.58,37.50,22.92
8,Group stage: Matchday 3,48,3.583333,0.473542,0.526458,48,58.33,31.25,10.42
9,Group stage: Matchday 1,48,2.958333,0.495417,0.504583,48,43.75,33.33,22.92


In [5]:
-- TEAMS_WITH_MOST_COMEBACK_WINS
WITH MatchPredictions AS (
    SELECT 
        TEAM_NAME_HOME AS TEAM_NAME,
        PREDICTION_TEAM_HOME_WIN AS WIN_PROBABILITY,
        TEAM_HOME_SCORE,
        TEAM_AWAY_SCORE,
        CASE WHEN TEAM_HOME_SCORE > TEAM_AWAY_SCORE THEN 1 ELSE 0 END AS ACTUAL_WIN
    FROM SOCCER.TBL_UEFA_2020
    UNION ALL
    SELECT 
        TEAM_NAME_AWAY AS TEAM_NAME,
        PREDICTION_TEAM_AWAY_WIN AS WIN_PROBABILITY,
        TEAM_AWAY_SCORE,
        TEAM_HOME_SCORE,
        CASE WHEN TEAM_AWAY_SCORE > TEAM_HOME_SCORE THEN 1 ELSE 0 END AS ACTUAL_WIN
    FROM SOCCER.TBL_UEFA_2020
    UNION ALL
    SELECT 
        TEAM_NAME_HOME AS TEAM_NAME,
        PREDICTION_TEAM_HOME_WIN AS WIN_PROBABILITY,
        TEAM_HOME_SCORE,
        TEAM_AWAY_SCORE,
        CASE WHEN TEAM_HOME_SCORE > TEAM_AWAY_SCORE THEN 1 ELSE 0 END AS ACTUAL_WIN
    FROM SOCCER.TBL_UEFA_2021
    UNION ALL
    SELECT 
        TEAM_NAME_AWAY AS TEAM_NAME,
        PREDICTION_TEAM_AWAY_WIN AS WIN_PROBABILITY,
        TEAM_AWAY_SCORE,
        TEAM_HOME_SCORE,
        CASE WHEN TEAM_AWAY_SCORE > TEAM_HOME_SCORE THEN 1 ELSE 0 END AS ACTUAL_WIN
    FROM SOCCER.TBL_UEFA_2021
    UNION ALL
    SELECT 
        TEAM_NAME_HOME AS TEAM_NAME,
        PREDICTION_TEAM_HOME_WIN AS WIN_PROBABILITY,
        TEAM_HOME_SCORE,
        TEAM_AWAY_SCORE,
        CASE WHEN TEAM_HOME_SCORE > TEAM_AWAY_SCORE THEN 1 ELSE 0 END AS ACTUAL_WIN
    FROM SOCCER.TBL_UEFA_2022
    UNION ALL
    SELECT 
        TEAM_NAME_AWAY AS TEAM_NAME,
        PREDICTION_TEAM_AWAY_WIN AS WIN_PROBABILITY,
        TEAM_AWAY_SCORE,
        TEAM_HOME_SCORE,
        CASE WHEN TEAM_AWAY_SCORE > TEAM_HOME_SCORE THEN 1 ELSE 0 END AS ACTUAL_WIN
    FROM SOCCER.TBL_UEFA_2022
)

SELECT 
    TEAM_NAME,
    COUNT(*) AS TOTAL_MATCHES,
    SUM(ACTUAL_WIN) AS TOTAL_WINS,
    SUM(CASE WHEN WIN_PROBABILITY < 0.5 AND ACTUAL_WIN = 1 THEN 1 ELSE 0 END) AS COMEBACK_WINS,
    ROUND(SUM(CASE WHEN WIN_PROBABILITY < 0.5 AND ACTUAL_WIN = 1 THEN 1 ELSE 0 END) * 100.0 / NULLIF(SUM(ACTUAL_WIN), 0), 2) AS PERCENT_ComeBACK_WINS
FROM MatchPredictions
GROUP BY TEAM_NAME
HAVING SUM(ACTUAL_WIN) >= 5  -- Only include teams with at least 5 wins
ORDER BY COMEBACK_WINS DESC, PERCENT_COMEBACK_WINS DESC
LIMIT 10;

,TEAM_NAME,TOTAL_MATCHES,TOTAL_WINS,COMEBACK_WINS,PERCENT_COMEBACK_WINS
0,Real Madrid,24,16,9,56.25
1,Manchester City,13,8,7,87.50
2,Chelsea,19,11,7,63.64
3,Man City,10,6,5,83.33
4,Liverpool,20,14,5,35.71
5,Napoli,10,7,4,57.14
6,Benfica,17,8,4,50.00
7,Inter,14,8,4,50.00
8,Ajax,14,8,4,50.00
9,Bayern Munich,10,8,4,50.00


In [7]:
-- Modified for better visualization
WITH TeamGoals AS (
    SELECT TEAM_NAME_HOME AS TEAM, TEAM_HOME_SCORE AS GOALS, 'Home' AS LOCATION FROM SOCCER.TBL_UEFA_2020
    UNION ALL SELECT TEAM_NAME_HOME, TEAM_HOME_SCORE, 'Home' FROM SOCCER.TBL_UEFA_2021
    UNION ALL SELECT TEAM_NAME_HOME, TEAM_HOME_SCORE, 'Home' FROM SOCCER.TBL_UEFA_2022
    UNION ALL SELECT TEAM_NAME_AWAY, TEAM_AWAY_SCORE, 'Away' FROM SOCCER.TBL_UEFA_2020
    UNION ALL SELECT TEAM_NAME_AWAY, TEAM_AWAY_SCORE, 'Away' FROM SOCCER.TBL_UEFA_2021
    UNION ALL SELECT TEAM_NAME_AWAY, TEAM_AWAY_SCORE, 'Away' FROM SOCCER.TBL_UEFA_2022
)
SELECT 
    TEAM,
    LOCATION,
    SUM(GOALS) AS TOTAL_GOALS,
    COUNT(*) AS MATCHES_PLAYED,
    ROUND(SUM(GOALS)/COUNT(*), 2) AS AVG_GOALS_PER_MATCH
FROM TeamGoals
GROUP BY TEAM, LOCATION
ORDER BY TOTAL_GOALS DESC
LIMIT 20;

,TEAM,LOCATION,TOTAL_GOALS,MATCHES_PLAYED,AVG_GOALS_PER_MATCH
0,Liverpool,Away,29,10,2.90
1,Real Madrid,Home,26,12,2.17
2,Manchester City,Home,25,7,3.57
3,Real Madrid,Away,24,12,2.00
4,Chelsea,Home,20,10,2.00
5,Benfica,Away,18,8,2.25
6,Benfica,Home,18,9,2.00
7,Ajax,Away,18,7,2.57
8,Man City,Home,17,6,2.83
9,Bayern,Home,16,4,4.00


In [8]:
-- Simple visualization-ready query
SELECT 
    STAGE,
    AVG(TEAM_HOME_SCORE + TEAM_AWAY_SCORE) AS AVG_GOALS,
    COUNT(*) AS MATCHES
FROM (
    SELECT STAGE, TEAM_HOME_SCORE, TEAM_AWAY_SCORE FROM SOCCER.TBL_UEFA_2020
    UNION ALL SELECT STAGE, TEAM_HOME_SCORE, TEAM_AWAY_SCORE FROM SOCCER.TBL_UEFA_2021
    UNION ALL SELECT STAGE, TEAM_HOME_SCORE, TEAM_AWAY_SCORE FROM SOCCER.TBL_UEFA_2022
)
GROUP BY STAGE
ORDER BY 
    CASE 
        WHEN STAGE LIKE 'Group%' THEN 1
        WHEN STAGE LIKE 'Round of%' THEN 2
        WHEN STAGE LIKE 'Quarter%' THEN 3
        WHEN STAGE LIKE 'Semi%' THEN 4
        WHEN STAGE LIKE 'Final%' THEN 5
        ELSE 6
    END;

,STAGE,AVG_GOALS,MATCHES
0,Group stage: Matchday 6,4.333333,6
1,Final,1.000000,4
2,Group stage: Matchday 4,3.125000,48
3,Group stage: Matchday 6,3.000000,42
4,Quarter-finals second leg,2.500000,12
5,Semi-finals second leg,3.000000,6
6,Round of 16 second leg,2.750000,24
7,Quarter-finals first leg,2.666667,12
8,Group stage: Matchday 1,2.958333,48
9,Group stage: Matchday 5,3.062500,48
